In [8]:
import os
import pandas as pd
import numpy as np
from glob import glob

from data_mani.utils import merge_market_and_gtrends
from prediction.models import RandomForestWrapper
from prediction.functions import get_selected_features, get_features_granger_huang
from prediction.functions import add_shift, annualy_fit_and_predict

In [2]:
# df = pd.read_csv("results/feature_selection/huang/indices/SPX Utilities.csv")
# # df = df.dropna()
# # print(df.shape)
# # df.head(9)

In [14]:
tickers = glob("data/indices/*.csv")
tickers = [n.split("/")[-1].split(".")[0] for n in tickers]

In [23]:
for ticker_name in tickers:

    # ticker_name = "SPX Utilities"
    path_list=["data", "gtrends.csv"]
#     fs_method = "huang"
    fs_method = "granger"
    select = get_features_granger_huang(ticker_name=ticker_name,
                                        out_folder="indices",
                                        fs_method=fs_method,
                                        path_list=path_list)
    print(ticker_name, len(select))

SPX Index 67
SPX Utilities 45
RTY Index 51
SPX Communications 72
SPX Consumer Cyclical 41
CCMP Index 64
SPX Energy 87
SPX Consumer Non cyclical 82
SPX Industrial 951
SPX Technology 65
SPX Financial 81
SPX Basic Materials 61


In [21]:
select

['banking_1',
 'banking_4',
 'fond_5',
 'invest_3',
 'kentucky_3',
 'markets_4',
 'stocks_13',
 'transaction_4',
 'transaction_7']

In [ ]:
ticker_name = "SPX Utilities"
market_folder = "toy"
path_list=["data", "gtrends.csv"]
path_t_list = [path_list[0],
               "src", "data",
               market_folder,
               "{}.csv".format(ticker_name)]
ticker_path = os.path.join(*path_t_list)
ticker_path = os.path.join(*["data",
                            market_folder,
                            "{}.csv".format(ticker_name)])

max_lag = 20
verbose = True
target_name = "target_return"

In [ ]:
train, test = merge_market_and_gtrends(ticker_path, test_size=0.5)
words = train.drop(target_name, 1).columns.to_list()
complete = pd.concat([train, test])
del train, test
add_shift(merged_df=complete,
              words=words,
              max_lag=max_lag,
              verbose=verbose)
complete = complete.fillna(0.0)
all_features = complete.drop(words + [target_name], 1).columns.to_list()

In [ ]:
market_folder = "indices"
fs_method = "sfi"
fs_method = "mdi"
fs_method = "mda"
select = get_selected_features(ticker_name=ticker_name,
                               out_folder="indices",
                               fs_method=fs_method,
                               path_list=path_list)
complete_selected = complete[[target_name] + select]
assert 2 < complete_selected.shape[1] < 3641

In [ ]:
Wrapper = RandomForestWrapper
n_iter = 1
n_jobs = 2
n_splits = 2
pred_results = annualy_fit_and_predict(df=complete_selected,
                                       Wrapper=Wrapper,
                                       n_iter=n_iter,
                                       n_jobs=n_jobs,
                                       n_splits=n_splits,
                                       target_name=target_name,
                                       verbose=False)

assert pred_results.shape[0] == complete_selected.loc["2005":].shape[0]